# **Scrapping Iran Exchange rate**
Nov 25, 2025

# Azad Rate

In [45]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://api.accessban.com/v1/market/indicator/summary-table-data/price_dollar_rl"

# Base query params copied from the site (you can simplify later)
query = {
    "lang": "fa",
    "order_dir": ["asc", ""],
    "draw": "9",
    "columns[0][data]": "0",
    "columns[0][name]": "",
    "columns[0][searchable]": "true",
    "columns[0][orderable]": "true",
    "columns[0][search][value]": "",
    "columns[0][search][regex]": "false",
    "columns[1][data]": "1",
    "columns[1][name]": "",
    "columns[1][searchable]": "true",
    "columns[1][orderable]": "true",
    "columns[1][search][value]": "",
    "columns[1][search][regex]": "false",
    "columns[2][data]": "2",
    "columns[2][name]": "",
    "columns[2][searchable]": "true",
    "columns[2][orderable]": "true",
    "columns[2][search][value]": "",
    "columns[2][search][regex]": "false",
    "columns[3][data]": "3",
    "columns[3][name]": "",
    "columns[3][searchable]": "true",
    "columns[3][orderable]": "true",
    "columns[3][search][value]": "",
    "columns[3][search][regex]": "false",
    "columns[4][data]": "4",
    "columns[4][name]": "",
    "columns[4][searchable]": "true",
    "columns[4][orderable]": "true",
    "columns[4][search][value]": "",
    "columns[4][search][regex]": "false",
    "columns[5][data]": "5",
    "columns[5][name]": "",
    "columns[5][searchable]": "true",
    "columns[5][orderable]": "true",
    "columns[5][search][value]": "",
    "columns[5][search][regex]": "false",
    "columns[6][data]": "6",
    "columns[6][name]": "",
    "columns[6][searchable]": "true",
    "columns[6][orderable]": "true",
    "columns[6][search][value]": "",
    "columns[6][search][regex]": "false",
    "columns[7][data]": "7",
    "columns[7][name]": "",
    "columns[7][searchable]": "true",
    "columns[7][orderable]": "true",
    "columns[7][search][value]": "",
    "columns[7][search][regex]": "false",
    "start": "0",          # offset (we'll change this in the loop)
    "length": "30",        # rows per page
    "search": "",
    "order_col": "",
    "from": "",
    "to": "",
    "convert_to_ad": "1",
}

all_rows = []
page = 0

while True:
    query["start"] = page * int(query["length"])
    r = requests.get(url, params=query)
    data = r.json()
    rows = data.get("data", [])

    if not rows:      # no more pages
        break

    all_rows.extend(rows)
    page += 1

# Build DataFrame
df = pd.DataFrame(all_rows)

# Column 4 and 5 are HTML snippets; strip tags
df[4] = df[4].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
df[5] = df[5].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())

# Optional: rename columns to something meaningful
df.columns = [
    "open", "low", "high", "close",
    "change_value", "change_percent",
    "date_gregorian", "date_jalali"
]
df.to_csv("azad_history_tgju.csv", index=False)
df.head()

,open,low,high,close,change_value,change_percent,date_gregorian,date_jalali
0,"1,133,150","1,127,800","1,138,200","1,134,600",3000,0.26%,2025/11/25,1404/09/04
1,"1,127,350","1,125,800","1,138,200","1,137,600",8500,0.75%,2025/11/23,1404/09/02
2,"1,135,100","1,127,800","1,135,200","1,129,100",6100,0.54%,2025/11/22,1404/09/01
3,"1,134,750","1,132,800","1,141,500","1,135,200",1600,0.14%,2025/11/20,1404/08/29
4,"1,126,700","1,117,600","1,134,500","1,133,600",8100,0.72%,2025/11/19,1404/08/28


In [49]:
# 1- Monthly average in geregorian calendar

df_az_g = df.copy()
df_az_g['date_gregorian'] = pd.to_datetime(df_az_g['date_gregorian'], format='%Y/%m/%d', errors='coerce')
df_az_g['year'] = df_az_g['date_gregorian'].dt.year
df_az_g['month'] = df_az_g['date_gregorian'].dt.month
df_az_g['day'] = df_az_g['date_gregorian'].dt.day

# Remove commas and convert the 'close' column to numeric
df_az_g['close'] = df_az_g['close'].str.replace(',', '').astype(float)
df_az_g['toman_close']= round(df_az_g['close'] / 10, 0)

df_az_g_m = df_az_g.groupby(['year','month']).agg({'toman_close': 'mean'}).reset_index()
df_az_g_m['date'] = df_az_g_m['year'].astype(str) + '-' + df_az_g_m['month'].astype(str).str.zfill(2)
df_az_g_m = df_az_g_m[['date', 'toman_close']].copy()

df_az_g_m.head()
df_az_g_m.to_csv('iran_exchange_gregorian_monthly_avg.csv', index=False)


In [51]:
# 2- Monthly average in jalali calendar

df_az_j = df.copy()

date_parts = df_az_j['date_jalali'].str.split('/', expand=True)
df_az_j['year'] = date_parts[0].astype(int)
df_az_j['month'] = date_parts[1].astype(int)
df_az_j['day'] = date_parts[2].astype(int)

# Remove commas and convert the 'close' column to numeric
df_az_j['close'] = df_az_j['close'].str.replace(',', '').astype(float)
df_az_j['toman_close']= round(df_az_j['close'] / 10, 0)

df_az_j_m = df_az_j.groupby(['year','month']).agg({'toman_close': 'mean'}).reset_index()
df_az_j_m['date'] = df_az_j_m['year'].astype(str) + '-' + df_az_j_m['month'].astype(str).str.zfill(2)
df_az_j_m = df_az_j_m[['date', 'toman_close']].copy()

df_az_j_m.head()
df_az_j_m.to_csv('iran_exchange_jalali_monthly_avg.csv', index=False)

# Nima Rate

In [52]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://api.accessban.com/v1/market/indicator/summary-table-data/nima_buy_usd"

# Base query params copied from the site (you can simplify later)
query = {
    "lang": "fa",
    "order_dir": ["asc", ""],
    "draw": "9",
    "columns[0][data]": "0",
    "columns[0][name]": "",
    "columns[0][searchable]": "true",
    "columns[0][orderable]": "true",
    "columns[0][search][value]": "",
    "columns[0][search][regex]": "false",
    "columns[1][data]": "1",
    "columns[1][name]": "",
    "columns[1][searchable]": "true",
    "columns[1][orderable]": "true",
    "columns[1][search][value]": "",
    "columns[1][search][regex]": "false",
    "columns[2][data]": "2",
    "columns[2][name]": "",
    "columns[2][searchable]": "true",
    "columns[2][orderable]": "true",
    "columns[2][search][value]": "",
    "columns[2][search][regex]": "false",
    "columns[3][data]": "3",
    "columns[3][name]": "",
    "columns[3][searchable]": "true",
    "columns[3][orderable]": "true",
    "columns[3][search][value]": "",
    "columns[3][search][regex]": "false",
    "columns[4][data]": "4",
    "columns[4][name]": "",
    "columns[4][searchable]": "true",
    "columns[4][orderable]": "true",
    "columns[4][search][value]": "",
    "columns[4][search][regex]": "false",
    "columns[5][data]": "5",
    "columns[5][name]": "",
    "columns[5][searchable]": "true",
    "columns[5][orderable]": "true",
    "columns[5][search][value]": "",
    "columns[5][search][regex]": "false",
    "columns[6][data]": "6",
    "columns[6][name]": "",
    "columns[6][searchable]": "true",
    "columns[6][orderable]": "true",
    "columns[6][search][value]": "",
    "columns[6][search][regex]": "false",
    "columns[7][data]": "7",
    "columns[7][name]": "",
    "columns[7][searchable]": "true",
    "columns[7][orderable]": "true",
    "columns[7][search][value]": "",
    "columns[7][search][regex]": "false",
    "start": "0",          # offset (we'll change this in the loop)
    "length": "30",        # rows per page
    "search": "",
    "order_col": "",
    "from": "",
    "to": "",
    "convert_to_ad": "1",
}

all_rows = []
page = 0

while True:
    query["start"] = page * int(query["length"])
    r = requests.get(url, params=query)
    data = r.json()
    rows = data.get("data", [])

    if not rows:      # no more pages
        break

    all_rows.extend(rows)
    page += 1

# Build DataFrame
df_n = pd.DataFrame(all_rows)

# Column 4 and 5 are HTML snippets; strip tags
df_n[4] = df_n[4].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
df_n[5] = df_n[5].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())

# Optional: rename columns to something meaningful
df_n.columns = [
    "open", "low", "high", "close",
    "change_value", "change_percent",
    "date_gregorian", "date_jalali"
]

df_n.to_csv("nima_history_tgju.csv", index=False)
df_n.head()

,open,low,high,close,change_value,change_percent,date_gregorian,date_jalali
0,"709,298","709,298","709,298","709,298",2464,0.35%,2025/11/25,1404/09/04
1,"711,762","711,762","711,762","711,762",1296,0.18%,2025/11/23,1404/09/02
2,"710,466","710,466","710,466","710,466",2272,0.32%,2025/11/20,1404/08/29
3,"708,194","708,194","708,194","708,194",268,0.04%,2025/11/19,1404/08/28
4,"707,926","707,926","707,926","707,926",451,0.06%,2025/11/18,1404/08/27


In [55]:
# 1- Monthly average in geregorian calendar

df_n_g = df_n.copy()
df_n_g['date_gregorian'] = pd.to_datetime(df_n_g['date_gregorian'], format='%Y/%m/%d', errors='coerce')
df_n_g['year'] = df_n_g['date_gregorian'].dt.year
df_n_g['month'] = df_n_g['date_gregorian'].dt.month
df_n_g['day'] = df_n_g['date_gregorian'].dt.day

# Remove commas and convert the 'close' column to numeric
df_n_g['close'] = df_n_g['close'].str.replace(',', '').astype(float)
df_n_g['nima_toman_close']= round(df_n_g['close'] / 10, 0)

df_n_g_m = df_n_g.groupby(['year','month']).agg({'nima_toman_close': 'mean'}).reset_index()
df_n_g_m['date'] = df_n_g_m['year'].astype(str) + '-' + df_n_g_m['month'].astype(str).str.zfill(2)
df_n_g_m = df_n_g_m[['date', 'nima_toman_close']].copy()

df_n_g_m.head()
df_n_g_m.to_csv('iran_nima_exchange_gregorian_monthly_avg.csv', index=False)

In [56]:
# 2- Monthly average in jalali calendar

df_n_j = df_n.copy()

date_n_parts = df_n_j['date_jalali'].str.split('/', expand=True)
df_n_j['year'] = date_n_parts[0].astype(int)
df_n_j['month'] = date_n_parts[1].astype(int)
df_n_j['day'] = date_n_parts[2].astype(int)

# Remove commas and convert the 'close' column to numeric
df_n_j['close'] = df_n_j['close'].str.replace(',', '').astype(float)
df_n_j['nima_toman_close']= round(df_n_j['close'] / 10, 0)

df_n_j_m = df_n_j.groupby(['year','month']).agg({'nima_toman_close': 'mean'}).reset_index()
df_n_j_m['date'] = df_n_j_m['year'].astype(str) + '-' + df_n_j_m['month'].astype(str).str.zfill(2)
df_n_j_m = df_n_j_m[['date', 'nima_toman_close']].copy()

df_n_j_m.head()
df_n_j_m.to_csv('iran_nima_exchange_jalali_monthly_avg.csv', index=False)